In [ ]:
#run this twice to load in the xlsx with all seasons and the 2018-19 csv
#reason they are different are entirely arbitrary
from google.colab import files
uploaded = files.upload()

Saving 2018-19.csv to 2018-19.csv


In [ ]:
#these are the imports needed
import tensorflow as tf
import numpy as np
import pandas as pd

from pandas import ExcelWriter
from pandas import ExcelFile

In [ ]:
#run this cell to load practice and test data
practiceData = pd.read_excel("2012-13-2017-18.xlsx",sheet_name='Sheet1') 
testData = pd.read_csv("2018-19.csv") 

In [ ]:
#this dictionary is meant to be used to access awayTeamLastTen
#some teams have repeated entries because of relocations/rebranding
#New Jersey -> Brooklyn Nets and New Orleans Hornets - > Pelicans. Charlotte changed from Bobcats to hornets but doesnt change their letters
teamDic = {"ATL":0,"BKN":1,"NJN":1,"BOS":2,"CHA":3,"CHI":4,"CLE":5,"DAL":6,"DEN":7,"DET":8,"GSW":9,"HOU":10,"IND":11,"LAC":12,"LAL":13,"MEM":14,"MIA":15,"MIL":16,"MIN":17,"NOP":18,"NOH":18,"NYK":19,"OKC":20,"ORL":21,"PHI":22,"PHX":23,"POR":24,"SAC":25,"SAS":26,"TOR":27,"UTA":27,"WAS":29}

#(total home wins/ total homes games played)
homeTeamWLAtHome = dict.fromkeys(teamDic,"0/0")

#list of queues that holds the last 10 away games for the away team in terms of W:1, L:0
#point of this is to use float(sum()/len()) to find win rate of last 10 away games for the away team
#[1,1,1,0,0,0,0,0,0,0] = 0.3
awayTeamLastTen = [[] for i in range(30)]

#this is to calculate average point differential for both team (total/numberofgamesteamhasplayed)
numberOfGamesTeamHasPlayed = dict.fromkeys(teamDic, 0)
totalPD = dict.fromkeys(teamDic,0)

In [ ]:
#this function takes in the dataframes to return the appropriate features and tensors
def ExtractData(insertedData):
  
  data = insertedData

  #we insert tensors one row at a time into these list

  #features
  tempTensor = []
  #labels
  tempLabels = []

  #something important to note about this loop
  #two rows are 1 game, one for the away team one for the home team
  #somethings are updated every iteration, somethings every second
  for index, row in data.iterrows():

      #breaks up 'GSW @ NOP' into ['GSW','@','NOP']
      #matcup[0]&[2] hold the teams, [1] holds the orientation (home/away) for the team of that row
      matchup = row['MATCHUP'].split(" ")

      #updates the total Point Differential for a team at that point in the season
      totalPD[matchup[0]] += row['PLUS_MINUS']

      #home team
      if matchup[1] == "vs.":   
          
          #these basically take in a "fraction" in a string and update it
          #adjust win rate for home team win
          if row['PLUS_MINUS'] > 0:
            temp = homeTeamWLAtHome[matchup[0]]
            temp = temp.split("/")
            temp = str((int(temp[0])+1)) + "/" + str((int(temp[1])+1))         
            homeTeamWLAtHome[matchup[0]] = temp

          #adjust for loss
          elif row['PLUS_MINUS'] < 0:
            temp = homeTeamWLAtHome[matchup[0]]
            temp = temp.split("/")          
            temp = str((int(temp[0]))) + "/" + str((int(temp[1])+1))        
            homeTeamWLAtHome[matchup[0]] = temp
          
      #away team. here i do list operations
      #tracks results of last 10 away games
      elif matchup[1] == "@":
        if row['PLUS_MINUS'] > 0:
          
          #win. add 1
          #if there is less than 10 away games played we dont pop
          if len(awayTeamLastTen[teamDic[matchup[0]]]) > 9:
            awayTeamLastTen[teamDic[matchup[0]]].pop(0)
            awayTeamLastTen[teamDic[matchup[0]]].append(1)
          else:
            awayTeamLastTen[teamDic[matchup[0]]].append(1)

        #add zeros for losing a game
        elif row['PLUS_MINUS'] < 0:
          if len(awayTeamLastTen[teamDic[matchup[0]]]) > 9:
            awayTeamLastTen[teamDic[matchup[0]]].pop(0)
            awayTeamLastTen[teamDic[matchup[0]]].append(0)
          
          else:
            awayTeamLastTen[teamDic[matchup[0]]].append(0)
      
      #2rows is 1 game
      if index % 2 == 0:
        numberOfGamesTeamHasPlayed[matchup[0]] += 1
        numberOfGamesTeamHasPlayed[matchup[2]] += 1
      
      # format of the tensor features
      # |home team|away team| H.T win % @ H| A.T win % @ last 10 A| H.T avg PD | A.T avg PD
      else:

        homeTeam =""
        awayTeam =""
        didHomeTeamWin = -1

        #we use these if statements to figure out if the home team since if vs. its [0] and if its @ its [2]
        if matchup[1] == "@":
          #WL is respective to matchup 0
          awayTeam = matchup[0]
          homeTeam = matchup[2]
          
          if row['WL'] == 'L':
            didHomeTeamWin = 1
          else:
            didHomeTeamWin = 0
        else:
          awayTeam = matchup[2]
          homeTeam = matchup[0]

          if row['WL'] == 'W':
            didHomeTeamWin = 1
          else:
            didHomeTeamWin = 0


        homeTeamWinPercentAtHome = homeTeamWLAtHome[homeTeam].split("/")
        homeTeamWinPercentAtHome = float( int(homeTeamWinPercentAtHome[0])/int(homeTeamWinPercentAtHome[1])) 

        #this is to prevent 0/0 error
        if len(awayTeamLastTen[teamDic[awayTeam]]) == 0:
          awayTeamWinPercentForLastTen = 0.0
        else:
          awayTeamWinPercentForLastTen = float ( sum(awayTeamLastTen[teamDic[awayTeam]])/ len(awayTeamLastTen[teamDic[awayTeam]]) )
        
        
        homePD = float(totalPD[homeTeam]/numberOfGamesTeamHasPlayed[homeTeam])
        awayPD = float(totalPD[awayTeam]/numberOfGamesTeamHasPlayed[awayTeam])

        #add to the list
        tempTensor.append( tf.constant( [teamDic[homeTeam],teamDic[awayTeam],homeTeamWinPercentAtHome,awayTeamWinPercentForLastTen,homePD,awayPD] ) )
        tempLabels.append(tf.constant([didHomeTeamWin]))

  #this converts the list of 1D tensors into one tensor
  tempTensor = tf.stack(tempTensor)
  tempLabels = tf.stack(tempLabels)

  return tempTensor,tempLabels

In [ ]:
practiceTensor, practiceLabels = ExtractData(practiceData)
testTensor, testLabels = ExtractData(testData)

In [ ]:
#feel free to mess around with this cell to see if you can improve the model
from tensorflow.keras import layers
model = tf.keras.Sequential([
  layers.Dense(6,input_dim=6,activation="relu"),
  layers.Dense(4,activation="relu"),
  layers.Dense(2,activation="relu"),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['binary_accuracy'])

model.fit(practiceTensor, practiceLabels, epochs=1000)
print("Done")

Epoch 1/1000
231/231 [==============================] - 1s 1ms/step - loss: 2.7619 - binary_accuracy: 0.5591
Epoch 2/1000
231/231 [==============================] - 0s 1ms/step - loss: 1.1997 - binary_accuracy: 0.5184
Epoch 3/1000
231/231 [==============================] - 0s 1ms/step - loss: 0.8939 - binary_accuracy: 0.5676
Epoch 4/1000
231/231 [==============================] - 0s 1ms/step - loss: 0.7490 - binary_accuracy: 0.5741
Epoch 5/1000
231/231 [==============================] - 0s 1ms/step - loss: 0.7136 - binary_accuracy: 0.5789
Epoch 6/1000
231/231 [==============================] - 0s 1ms/step - loss: 0.7006 - binary_accuracy: 0.5808
Epoch 7/1000
231/231 [==============================] - 0s 1ms/step - loss: 0.6950 - binary_accuracy: 0.5840
Epoch 8/1000
231/231 [==============================] - 0s 1ms/step - loss: 0.6907 - binary_accuracy: 0.5862
Epoch 9/1000
231/231 [==============================] - 0s 1ms/step - loss: 0.6870 - binary_accuracy: 0.5886
Epoch 10/1000
231/2

In [ ]:
probability_model = tf.keras.Sequential([model])
predictions = probability_model.predict(testTensor)

#if prediction is right, a 1 is appened.
#if Wrong, a 0.
wasPredictionCorrect = []

for i in range(len(predictions)):
  #the extra [0] converts to a numpy array gets the 0th (only) element
  temp = -1
  if predictions[i][0] < 0.5:
    temp = 0
  else:
    temp = 1

  if temp == testLabels[i][0]:
    wasPredictionCorrect.append(1)
  else:
    wasPredictionCorrect.append(0)
  
  #print("prediction for",i," : ", predictions[i][0],", actual :",testLabels[i].numpy()[0])

print(float(sum(wasPredictionCorrect)/len(wasPredictionCorrect)))

0.6934959349593496
